In [14]:
import geopandas as gpd
import rasterio
import numpy as np

def crs_to_dict(crs):
    # Convert NumPy arrays to Python lists
    def convert(o):
        if isinstance(o, np.ndarray):
            return o.tolist()
        return o

    return {key: convert(value) for key, value in crs.to_dict().items()}

def verify_metadata(data, is_vector=True):
    metadata = {}

    # Common metadata
    metadata["crs"] = crs_to_dict(data.crs)
    metadata["extent"] = data.total_bounds.tolist()

    if is_vector:
        # Vector-specific metadata
        metadata["geometry_types"] = data.geom_type.value_counts().to_dict()

        # Additional metadata fields
        metadata["categories"] = ["Adm data"]
        metadata["keywords"] = ["Adm", "state"]
        metadata["access"] = {"level": "Public", "rights": "Read-only"}
        metadata["validation"] = {"status": "Validated", "method": "Manual validation"}
    else:
        # Raster-specific metadata
        with rasterio.open(data) as src:
            metadata["data_type"] = src.dtypes[0]

            # Additional metadata fields (replace these with actual metadata extraction logic)
            metadata["categories"] = ["Adm data"]
            metadata["keywords"] = ["Adm", "state"]
            metadata["access"] = {"level": "Public", "rights": "Read-only"}
            metadata["validation"] = {"status": "Validated", "method": "Manual validation"}
    return metadata


def store_metadata_in_database(data, metadata, limit=None):
    conn = psycopg2.connect(
    host='localhost',
    port='5433',
    user='postgres',
    password='postgres',
    database='postgres'
)
    cursor = conn.cursor(cursor_factory=RealDictCursor)

    # Create the table if it does not exist
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS geospatial_data (
            id SERIAL PRIMARY KEY,
            name VARCHAR(255),
            metadata JSONB
        )
    """)
    conn.commit()

    # Insert the first n records into the database
    for index, row in data.head(limit).iterrows():
        # Insert data along with metadata into the database
        cursor.execute("""
            INSERT INTO geospatial_data (name, metadata)
            VALUES (%s, %s)
        """, (metadata["categories"][0], psycopg2.extras.Json(metadata)))
    
    conn.commit()

# Example usage
input_file_path = "GeospatialDataMangement/Geojson1.geojson"
processed_data = gpd.read_file(input_file_path)
metadata = verify_metadata(processed_data)
store_metadata_in_database(processed_data, metadata, limit=100)


/Users/navjothbn/opt/anaconda3/lib/python3.9/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj = self._crs.to_proj4(version=version)


In [22]:
import psycopg2
from psycopg2.extras import RealDictCursor
import json

def fetch_metadata_by_id_from_database(record_id):
    conn = psycopg2.connect(
        host='localhost',
        port='5433',
        user='postgres',
        password='postgres',
        database='postgres'
    )
    cursor = conn.cursor(cursor_factory=RealDictCursor)

    cursor.execute("""
        SELECT metadata FROM geospatial_data WHERE id = %s
    """, (record_id,))
    
    metadata_records = cursor.fetchall()

    conn.close()

    if metadata_records:
        return metadata_records[0]['metadata']
    else:
        return None

user_record_id = 10 

fetched_metadata = fetch_metadata_by_id_from_database(user_record_id)

if fetched_metadata:
    print("Fetched Metadata:")
    print(json.dumps(fetched_metadata, indent=4))
else:
    print(f"No metadata found for record ID: {user_record_id}")


Fetched Metadata:
{
    "crs": {
        "proj": "longlat",
        "type": "crs",
        "datum": "WGS84",
        "no_defs": null
    },
    "access": {
        "level": "Public",
        "rights": "Read-only"
    },
    "extent": [
        68.5393,
        7.0045,
        96.8204,
        34.9946
    ],
    "keywords": [
        "Adm",
        "state"
    ],
    "categories": [
        "Adm data"
    ],
    "validation": {
        "method": "Manual validation",
        "status": "Validated"
    },
    "geometry_types": {
        "Point": 75089
    }
}
